In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from func import *

In [2]:
def spectroastrometry(mini_cube,mini_err,mini_wave,fixed_param,MC_loops,broad2=False):
    output_par = np.zeros((10,mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    output_par_err = np.zeros((10,mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    fitted = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    residual = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    if broad2:
        full_gauss_fixkin = full_gauss2_fixkin
    else:
        full_gauss_fixkin = full_gauss1_fixkin
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])    
            popt_full_fit,pcov_full_fit = leastsq(full_gauss_fixkin,x0=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, -0.7, 0.001],args=(mini_wave,spectrum,error,fixed_param),maxfev = 10000000)
            
            output_par[:,i,j]=popt_full_fit
            fitted[:,i,j]=(full_gauss_fixkin(popt_full_fit,mini_wave,spectrum,error,fixed_param))*(error)+spectrum 
            residual[:,i,j] = mini_cube[:,i,j] - fitted[:,i,j]
            
            spec_parameters_MC = np.zeros((len(popt_full_fit),MC_loops))
            for l in range(MC_loops):
                iteration_data = np.random.normal(spectrum,error)   
                popt_spec_MC,pcov_spec_MC = leastsq(full_gauss_fixkin,x0=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, -0.7, 0.001],args=(mini_wave,iteration_data,error,fixed_param),maxfev = 10000000)
                spec_parameters_MC[:,l]=popt_spec_MC
                
            spec_parameters_err = np.std(spec_parameters_MC,1)
            output_par_err[:,i,j] = spec_parameters_err
    return output_par,output_par_err,fitted,residual

def flux_map(output_par,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(output_par[0,:,:],name='amp_Hb'))
    hdus.append(fits.ImageHDU(output_par[1,:,:],name='amp_OIII5007'))
    hdus.append(fits.ImageHDU(output_par[2,:,:],name='amp_OIII5007_br'))
    hdus.append(fits.ImageHDU(output_par[3,:,:],name='amp_Hb_br'))
    hdus.append(fits.ImageHDU(output_par[4,:,:],name='amp_Hb1'))
    hdus.append(fits.ImageHDU(output_par[5,:,:],name='amp_Hb2'))
    hdus.append(fits.ImageHDU(output_par[6,:,:],name='amp_Fe5018_1'))
    hdus.append(fits.ImageHDU(output_par[7,:,:],name='amp_Fe5018_2'))
    hdus.append(fits.ImageHDU(output_par[8,:,:],name='m'))
    hdus.append(fits.ImageHDU(output_par[9,:,:],name='c'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')

def flux_map_err(output_par_err,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(output_par_err[0,:,:],name='amp_Hb_err'))
    hdus.append(fits.ImageHDU(output_par_err[1,:,:],name='amp_OIII5007_err'))
    hdus.append(fits.ImageHDU(output_par_err[2,:,:],name='amp_OIII5007_br_err'))
    hdus.append(fits.ImageHDU(output_par_err[3,:,:],name='amp_Hb_br_err'))
    hdus.append(fits.ImageHDU(output_par_err[4,:,:],name='amp_Hb1_err'))
    hdus.append(fits.ImageHDU(output_par_err[5,:,:],name='amp_Hb2_err'))
    hdus.append(fits.ImageHDU(output_par_err[6,:,:],name='amp_Fe5018_1_err'))
    hdus.append(fits.ImageHDU(output_par_err[7,:,:],name='amp_Fe5018_2_err'))
    hdus.append(fits.ImageHDU(output_par_err[8,:,:],name='m_err'))
    hdus.append(fits.ImageHDU(output_par_err[9,:,:],name='c_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')


In [3]:
def spectroastrometric_script(obj,z,broad2,MC_loops=100,prefix_path_cube="/home/mainak/ftp.hidrive.strato.com/users/login-carsftp/IFU_data",destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    fixed_param = fixed_parameters(obj)
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s/%s.9_arcsec_minicube.fits'%(destination_path_cube,obj,obj))
 
    (output_par,output_par_err,fitted_cube,residual_cube) = spectroastrometry(mini_cube,mini_err,mini_wave,fixed_param,MC_loops,broad2)
    
    store_cube('%s/%s/%s.9_arcsec_fitted_residual_minicube.fits'%(destination_path_cube,obj,obj),fitted_cube,mini_wave,residual_cube,mini_header)    
    flux_map(output_par,obj)
    flux_map_err(output_par_err,obj)
    
    (centroid_OIII_nr,centroid_OIII_br,centroid_Hb_blr_br) = light_weighted_centroid(obj)
    print ('%s.fits'%(obj))

In [4]:
z = {"HE0108-4743":0.02392}

objs = z.keys()

broad2= {'HE0108-4743':True}

for obj in objs:
    spectroastrometric_script(obj,z[obj],broad2[obj])

HE0108-4743.fits


In [4]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':True}

for obj in objs:
    spectroastrometric_script(obj,z[obj],broad2[obj])

HE0021-1810.fits
HE0021-1819.fits
HE0040-1105.fits
HE0108-4743.fits
HE0114-0015.fits
HE0119-0118.fits
HE0212-0059.fits
HE0224-2834.fits
HE0227-0913.fits
HE0232-0900.fits
HE0253-1641.fits
HE0345+0056.fits
HE0351+0240.fits
HE0412-0803.fits
HE0429-0247.fits
HE0433-1028.fits
HE0853+0102.fits
HE0934+0119.fits
HE1011-0403.fits
HE1017-0305.fits
HE1029-1831.fits
HE1107-0813.fits
HE1108-2813.fits
HE1126-0407.fits
HE1237-0504.fits
HE1248-1356.fits
HE1330-1013.fits
HE1353-1917.fits
HE1417-0909.fits
HE2128-0221.fits
HE2211-3903.fits
HE2222-0026.fits
HE2233+0124.fits
HE2302-0857.fits
